In [ ]:
import sys
sys.path.append('../../')
import pandas as pd
from SynTemp.SynUtils.utils import load_database, save_database, load_from_pickle

nh_r0_fw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/fw_0.json.gz')
nh_r0_bw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/bw_0.json.gz')
nh_r1_fw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/fw_1.json.gz')
nh_r1_bw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/bw_1.json.gz')
nh_r2_fw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/fw_2.json.gz')
nh_r2_bw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/bw_2.json.gz')
nh_r3_fw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/fw_3.json.gz')
nh_r3_bw = load_database('../../Data/DPO/USPTO_50K/Non_hydrogen/Output/bw_3.json.gz')

h_r0_fw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/fw_0.json.gz')
h_r0_bw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/bw_0.json.gz')
h_r1_fw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/fw_1.json.gz')
h_r1_bw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/bw_1.json.gz')
h_r2_fw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/fw_2.json.gz')
h_r2_bw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/bw_2.json.gz')
h_r3_fw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/fw_3.json.gz')
h_r3_bw = load_database('../../Data/DPO/USPTO_50K/Hydrogen/Output/bw_3.json.gz')

gh_r0_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_0.json.gz')
gh_r0_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_0.json.gz')
gh_r1_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_1.json.gz')
gh_r1_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_1.json.gz')
gh_r2_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_2.json.gz')
gh_r2_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_2.json.gz')
gh_r3_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_3.json.gz')
gh_r3_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_3.json.gz')



hier_r0_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_hier_0.json.gz')
hier_r0_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_hier_0.json.gz')
hier_r1_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_hier_1.json.gz')
hier_r1_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_hier_1.json.gz')
hier_r2_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_hier_2.json.gz')
hier_r2_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_hier_2.json.gz')
hier_r3_fw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/fw_hier_3.json.gz')
hier_r3_bw = load_database('../../Data/DPO/USPTO_50K/Good_hydrogen/Output/bw_hier_3.json.gz')

# 1. Coverage analysis

In [ ]:
from typing import List, Dict, Tuple
import numpy as np

def reconstruction_rate_solutions(data: List[Dict], solutions_column: str = 'unrank', positive_columns: str = 'positive_reactions') -> Tuple[float, float]:
    """
    Calculate the reconstruction rate and coverage rate for solutions in the data.

    Parameters:
    - data: List of dictionaries containing the data.
    - solutions_column: The key in the dictionaries representing the solutions. Default is 'unrank'.
    - positive_columns: The key in the dictionaries representing the positive reactions. Default is 'positive_reactions'.

    Returns:
    - A tuple containing:
      - The total solutions rate: total number of solutions divided by the number of entries in the data.
      - The coverage rate: the number of entries with positive reactions divided by the number of entries in the data.
    """
    if not data:
        return 0.0, 0.0

    total_solutions = []
    coverage = 0
    fpr = []
    for entry in data:
        solutions = []
        false_solutions = []
        if solutions_column in entry and isinstance(entry[solutions_column], list):
            #total_solutions += len(entry[solutions_column])
            solutions = false_solutions = len(entry[solutions_column])
            total_solutions.append(solutions)
        else:
            raise KeyError(f"Missing or invalid '{solutions_column}' in entry: {entry}")

        if positive_columns in entry and entry[positive_columns]:
            false_solutions -= 1
            coverage += 1
        if solutions > 0:
            fpr.append(round(false_solutions/solutions, 2))
        else:
            fpr.append(1)

    total_entries = len(data)

    total_solutions_mean = round(np.mean(total_solutions), 2)
    #total_solutions_std = round(np.std(total_solutions),2)
    coverage_rate = coverage / total_entries

    return total_solutions_mean, round(coverage_rate*100,2), round(np.mean(fpr),2)

In [ ]:
list_data_fw = [[nh_r0_fw, nh_r1_fw, nh_r2_fw, nh_r3_fw], 
                [h_r0_fw, h_r1_fw, h_r2_fw, h_r3_fw],
                [gh_r0_fw, gh_r1_fw, gh_r2_fw, gh_r3_fw],
                [hier_r0_fw, hier_r1_fw, hier_r2_fw, hier_r3_fw]]

data_list_dict_fw_cover = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

data_list_dict_fw_sol = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

data_list_dict_fw_fpr = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]


list_data_bw = [[nh_r0_bw, nh_r1_bw, nh_r2_bw, nh_r3_bw], 
                [h_r0_bw, h_r1_bw, h_r2_bw, h_r3_bw],
                [gh_r0_bw, gh_r1_bw, gh_r2_bw, gh_r3_bw],
                [hier_r0_bw, hier_r1_bw, hier_r2_bw, hier_r3_bw]]

data_list_dict_bw_cover = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

data_list_dict_bw_sol = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

data_list_dict_bw_fpr = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

In [ ]:
for key, data in enumerate(list_data_fw):
    for radius, value in enumerate(data):
        result = reconstruction_rate_solutions(value)
        data_list_dict_fw_cover[key][f'R{radius}'] = result[1]
        data_list_dict_fw_sol[key][f'R{radius}'] = result[0]
        data_list_dict_fw_fpr[key][f'R{radius}'] = result[2]

for key, data in enumerate(list_data_bw):
    for radius, value in enumerate(data):
        result = reconstruction_rate_solutions(value)
        data_list_dict_bw_cover[key][f'R{radius}'] = result[1]
        data_list_dict_bw_sol[key][f'R{radius}'] = result[0]
        data_list_dict_bw_fpr[key][f'R{radius}'] = result[2]

In [ ]:
fw_list_dict = []
for i in range(len(data_list_dict_fw_sol)):
    new_dict = {'Type':[], 'reconstruction_rates': [], 'number_of_solutions':[]}
    new_dict['Type'] = data_list_dict_fw_sol[i]['Type']
    new_dict['reconstruction_rates'].extend([data_list_dict_fw_cover[i][f'R{j}'] for j in range(4)])
    new_dict['number_of_solutions'].extend([data_list_dict_fw_fpr[i][f'R{j}'] for j in range(4)])
    fw_list_dict.append(new_dict)


In [ ]:
bw_list_dict = []
for i in range(len(data_list_dict_bw_sol)):
    new_dict = {'Type':[], 'reconstruction_rates': [], 'number_of_solutions':[]}
    new_dict['Type'] = data_list_dict_bw_sol[i]['Type']
    new_dict['reconstruction_rates'].extend([data_list_dict_bw_cover[i][f'R{j}'] for j in range(4)])
    new_dict['number_of_solutions'].extend([data_list_dict_bw_fpr[i][f'R{j}'] for j in range(4)])
    bw_list_dict.append(new_dict)


In [ ]:
fw_list_dict

In [ ]:
bw_list_dict[-2:-1]

In [ ]:
fw_list_dict

In [ ]:
def convert_data(data):
    # Initialize a dictionary to store the extracted data
    structured_data = {
        "Type": [],
        "Reconstruction Rate R0": [],
        "Reconstruction Rate R1": [],
        "Reconstruction Rate R2": [],
        "Reconstruction Rate R3": [],
        "Number of Solutions R0": [],
        "Number of Solutions R1": [],
        "Number of Solutions R2": [],
        "Number of Solutions R3": []
    }

    # Iterate through each item in the input data
    for item in data:
        # Append the type
        structured_data["Type"].append(item["Type"])
        
        # Append reconstruction rates for R0, R1, and R2
        structured_data["Reconstruction Rate R0"].append(item["reconstruction_rates"][0])
        structured_data["Reconstruction Rate R1"].append(item["reconstruction_rates"][1])
        structured_data["Reconstruction Rate R2"].append(item["reconstruction_rates"][2])
        structured_data["Reconstruction Rate R3"].append(item["reconstruction_rates"][3])
        
        # Append number of solutions for R0, R1, and R2
        structured_data["Number of Solutions R0"].append(item["number_of_solutions"][0])
        structured_data["Number of Solutions R1"].append(item["number_of_solutions"][1])
        structured_data["Number of Solutions R2"].append(item["number_of_solutions"][2])
        structured_data["Number of Solutions R3"].append(item["number_of_solutions"][3])

    return structured_data


In [ ]:
df_combined_fw =  pd.DataFrame(convert_data(fw_list_dict))
df_combined_bw =  pd.DataFrame(convert_data(bw_list_dict))

In [ ]:
import matplotlib.pyplot as plt

# Enable LaTeX rendering in matplotlib
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')  # Ensure amsmath is loaded

def plot_reconstruction_rate(df, ax, colors, title='A'):
    width = 0.2  # Adjusted width of the bars to fit four bars
    x = range(len(df["Type"]))

    # Plotting Reconstruction Rates with adjusted bar positions for clarity
    bars_r0 = ax.bar([pos - 1.5*width for pos in x], round(df["Reconstruction Rate R0"],0), width=width, label='R0', color=colors[0])
    bars_r1 = ax.bar([pos - 0.5*width for pos in x], round(df["Reconstruction Rate R1"],0), width=width, label='R1', color=colors[1])
    bars_r2 = ax.bar([pos + 0.5*width for pos in x], round(df["Reconstruction Rate R2"], 0), width=width, label='R2', color=colors[2])
    bars_r3 = ax.bar([pos + 1.5*width for pos in x], round(df["Reconstruction Rate R3"],0), width=width, label='R3', color=colors[3])

    # Adding numbers above bars with bold formatting for better visibility
    for bars in [bars_r0, bars_r1, bars_r2, bars_r3]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.0f}', 
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', 
                        fontweight='medium', fontsize=20)

    # Aesthetic enhancements
    ax.set_ylabel(r'Reconstruction Rate (\%)', fontsize=24, fontweight='medium')
    ax.set_title(title, fontsize=30, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df["Type"], rotation=45, fontsize=24)
    #ax.set_xticklabels([r'$Q_{\text{raw}}$', r'$Q_{\text{complete}}$', r'$Q_{\text{cyclic}}$', r'$Q_{\text{hier}}$'], rotation=45, fontsize=24)
    ax.set_yticks([0,20,40,60,80])
    ax.set_yticklabels([0,20,40,60,80], fontsize=20)
    ax.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')
    ax.set_axisbelow(True)

def plot_number_of_solutions(df, ax, colors, title='B'):
    width = 0.2  # Adjusted width for four bars
    x = range(len(df["Type"]))

    # Plotting Number of Solutions with consistent styling
    bars_r0 = ax.bar([pos - 1.5*width for pos in x], round(100*df["Number of Solutions R0"]), width=width, label='R0', color=colors[0])
    bars_r1 = ax.bar([pos - 0.5*width for pos in x], round(100*df["Number of Solutions R1"]), width=width, label='R1', color=colors[1])
    bars_r2 = ax.bar([pos + 0.5*width for pos in x], round(100*df["Number of Solutions R2"]), width=width, label='R2', color=colors[2])
    bars_r3 = ax.bar([pos + 1.5*width for pos in x], round(100*df["Number of Solutions R3"]), width=width, label='R3', color=colors[3])

    # Adding numbers above bars with bold formatting
    for bars in [bars_r0, bars_r1, bars_r2, bars_r3]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.0f}', 
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', 
                        fontweight='medium', fontsize=20)

    # Aesthetic enhancements
    ax.set_ylabel(r'False Possitive Rate (\%)', fontsize=24, fontweight='medium')
    ax.set_title(title, fontsize=30, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df["Type"], rotation=45, fontsize=24)
    #ax.set_xticklabels([r'$Q_{\text{raw}}$', r'$Q_{\text{complete}}$', r'$Q_{\text{cyclic}}$', r'$Q_{\text{hier}}$'], rotation=45, fontsize=24)
    ax.set_yticks([0,20,40,60,80,100])
    ax.set_yticklabels([0,20,40,60,80,100], fontsize=20)
    ax.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')
    ax.set_axisbelow(True)

# Define colors for R0, R1, R2
colors = ['#3A8EBA', '#92C5DE', '#F4A582', '#D6604D']


# Creating the figure and subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 18))

# Plotting data
plot_reconstruction_rate(df_combined_fw, ax1, colors, 'A')
plot_reconstruction_rate(df_combined_bw, ax2, colors, 'B')

plot_number_of_solutions(df_combined_fw, ax3, colors, 'C')
plot_number_of_solutions(df_combined_bw, ax4, colors, 'D')


fig.legend(['R0', 'R1', 'R2', 'R3'], loc='lower center', ncol=2, fontsize=20, bbox_to_anchor=(0.5, 0.001))

# Adjusting layout
fig.tight_layout()
fig.subplots_adjust(bottom=0.1) 
fig.savefig('./fig/template_false_rate_compare.pdf', dpi=600)

# 2. TopK accuracy random

In [ ]:
import random
import numpy as np
from joblib import Parallel, delayed
from SynTemp.SynRule.rule_benchmark import RuleBenchmark
from SynTemp.SynChemistry.sf_random import SFRandom
from typing import List, Dict, Tuple

def random_evaluation(data: List[Dict], ground_truth_key: str = 'reactions', rank_list_key: str = "ranked_reactions", k: int = 1, ignore_stereo: bool = True, n_eval: int = 10, n_jobs: int = 4) -> Tuple[float, float, List[float]]:
    """
    Evaluate the performance of a ranking system using random seeds in parallel.

    Parameters:
    - data: List of dictionaries containing the data.
    - ground_truth_key: The key for the ground truth reactions. Default is 'reactions'.
    - rank_list_key: The key for the ranked reactions list. Default is 'ranked_reactions'.
    - k: The top k rankings to consider for accuracy. Default is 1.
    - ignore_stereo: Boolean flag to ignore stereo chemistry. Default is True.
    - n_eval: Number of evaluations to perform. Default is 10.
    - n_jobs: The number of jobs to run in parallel. Default is -1 (use all available processors).

    Returns:
    - A tuple containing:
      - Mean accuracy of the evaluations.
      - Standard deviation of the accuracies.
      - List of individual evaluation results.
    """
    if not data:
        raise ValueError("Data list is empty.")

    def evaluate(seed: int) -> float:
        try:
            result = RuleBenchmark.TopKAccuracy(data, ground_truth_key, rank_list_key, k, ignore_stereo, scoring_function=SFRandom(seed))
        except Exception as e:
            raise RuntimeError(f"Error during evaluation with seed {seed}: {e}")
        return result

    seeds = [random.randint(0, 1000000) for _ in range(n_eval)]
    results = Parallel(n_jobs=n_jobs)(delayed(evaluate)(seed) for seed in seeds)

    mean_accuracy = round(np.mean(results),2)
    std_accuracy = round(np.std(results),2)

    return mean_accuracy, std_accuracy, results

In [ ]:
list_data_fw = [[nh_r0_fw, nh_r1_fw, nh_r2_fw, nh_r3_fw], 
                [h_r0_fw, h_r1_fw, h_r2_fw, h_r3_fw],
                [gh_r0_fw, gh_r1_fw, gh_r2_fw, gh_r3_fw],
                [hier_r0_fw, hier_r1_fw, hier_r2_fw, hier_r3_fw]]
data_list_dict_fw = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]
all_matrix_fw = []
for key, data in enumerate(list_data_fw):
    for radius, value in enumerate(data):
        result = random_evaluation(value, k = 10)
        data_list_dict_fw[key][f'R{radius}'] = f'{result[0]} ± {result[1]}'
        all_matrix_fw.append(result[2])

In [ ]:
list_data_bw = [[nh_r0_bw, nh_r1_bw, nh_r2_bw, nh_r3_bw], 
                [h_r0_bw, h_r1_bw, h_r2_bw, h_r3_bw],
                [gh_r0_bw, gh_r1_bw, gh_r2_bw, gh_r3_bw],
                [hier_r0_bw, hier_r1_bw, hier_r2_bw, hier_r3_bw]]

data_list_dict_bw = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]
all_matrix_bw = []
for key, data in enumerate(list_data_bw):
    for radius, value in enumerate(data):
        result = random_evaluation(value, k = 10)
        data_list_dict_bw[key][f'R{radius}'] = f'{result[0]} ± {result[1]}'
        all_matrix_bw.append(result[2])

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from typing import List

def statistical_comparison(data_lists: List[List[float]]) -> np.ndarray:
    """
    Perform statistical comparison among multiple lists with Bonferroni correction
    and return a matrix of corrected p-values.

    Parameters:
    - data_lists: A list of lists, where each sublist contains numerical data.

    Returns:
    - A matrix of corrected p-values representing pairwise comparisons.
    """
    num_lists = len(data_lists)
    p_value_matrix = np.zeros((num_lists, num_lists))
    num_comparisons = num_lists * (num_lists - 1) // 2  # Total number of comparisons

    for i in range(num_lists):
        for j in range(i, num_lists):
            if i == j:
                p_value_matrix[i, j] = np.nan
            else:
                _, p_value = ttest_ind(data_lists[i], data_lists[j], equal_var=False)
                corrected_p_value = np.round(p_value * num_comparisons, 3)
                # Ensure that corrected p-values do not exceed 1
                corrected_p_value = min(corrected_p_value, 1.0)
                p_value_matrix[i, j] = corrected_p_value
                p_value_matrix[j, i] = corrected_p_value

    return p_value_matrix

In [ ]:
fw_test = statistical_comparison(all_matrix_fw)
bw_test = statistical_comparison(all_matrix_bw)

In [ ]:
name = [r'$Q_{\text{raw}}-R_0$', r'$Q_{\text{raw}}-R_1$', r'$Q_{\text{raw}}-R_2$',r'$Q_{\text{raw}}-R_3$',
        r'$Q_{\text{complete}}-R_0$', r'$Q_{\text{complete}}-R_1$', r'$Q_{\text{complete}}-R_2$',r'$Q_{\text{complete}}-R_3$',
        r'$Q_{\text{cyclic}}-R_0$', r'$Q_{\text{cyclic}}-R_1$', r'$Q_{\text{cyclic}}-R_2$',r'$Q_{\text{cyclic}}-R_3$',
        r'$Q_{\text{hier}}-R_0$', r'$Q_{\text{hier}}-R_1$', r'$Q_{\text{hier}}-R_2$',r'$Q_{\text{hier}}-R_3$',
         ]

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

def plot_p_value_heatmap(p_value_matrix: np.ndarray, names: list):
    """
    Plot a heatmap for the p-value matrix with two distinct colors indicating p-values
    greater than 0.05 and less than or equal to 0.05, without a color bar but with an external legend
    including a legend for 'N/A' values.

    Parameters:
    - p_value_matrix: A matrix of p-values representing pairwise comparisons.
    - names: List of names corresponding to the matrix indices.
    """
    # Mask diagonal for N/A values
    np.fill_diagonal(p_value_matrix, -1)

    # Enhanced color palette for better contrast
    colors = ["#D3D3D3", "#A2CEE3", "#D6604D"]  # Blue for p <= 0.05, Red for p > 0.05, Gray for N/A
    cmap = ListedColormap(colors)
    boundaries = [-1, 0, 0.05, 1]  # Define boundaries for significance and N/A values
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)

    # Create the figure
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(p_value_matrix, annot=False, fmt=".2f", cmap=cmap, norm=norm,
                     xticklabels=names, yticklabels=names,
                     cbar=False,  
                     linewidths=.5, linecolor='black')

   
    plt.xticks(rotation=45, fontsize = 14)
    
    plt.yticks(rotation=0, fontsize = 14)
    plt.tight_layout()

    
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label=r'$p \leq 0.05$', markersize=10, markerfacecolor='#A2CEE3'),
        plt.Line2D([0], [0], marker='o', color='w', label=r'$p > 0.05$', markersize=10, markerfacecolor='#FA8072'),
        plt.Line2D([0], [0], marker='o', color='w', label='N/A', markersize=10, markerfacecolor='#D3D3D3')
    ]
    # Position the legend outside of the heatmap
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.02, 1), loc='upper left', prop={"size":16})

    plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

def plot_p_value_heatmap(p_value_matrix: np.ndarray, names: list, ax=None, title='A'):
    """
    Plot a heatmap for the p-value matrix with two distinct colors indicating p-values
    greater than 0.05 and less than or equal to 0.05, without a color bar but with an external legend
    including a legend for 'N/A' values.

    Parameters:
    - p_value_matrix: A matrix of p-values representing pairwise comparisons.
    - names: List of names corresponding to the matrix indices.
    - ax: Optional matplotlib axes object. If none is provided, one will be created.
    """
    # Mask diagonal for N/A values
    np.fill_diagonal(p_value_matrix, -1)

    # Enhanced color palette for better contrast
    colors = ["#D3D3D3", "#A2CEE3", "#D6604D"]  # Blue for p <= 0.05, Red for p > 0.05, Gray for N/A
    cmap = ListedColormap(colors)
    boundaries = [-1, 0, 0.05, 1]  # Define boundaries for significance and N/A values
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)

    # Check if an axes object is provided; if not, create a new figure and axes
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))

    sns.heatmap(p_value_matrix, annot=False, fmt=".2f", cmap=cmap, norm=norm,
                xticklabels=names, yticklabels=names,
                cbar=False, ax=ax,
                linewidths=.5, linecolor='black')
    ax.set_title(title, fontsize = 30, weight = 'bold')
    ax.set_xticklabels(names, rotation=45, fontsize=20)
    ax.set_yticklabels(names, rotation=0, fontsize=20)
    ax.figure.tight_layout()




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming plot_p_value_heatmap is defined elsewhere with the correct definition to accept 'ax'
# Set style
sns.set(style="whitegrid")

# Create figure and subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 10), dpi=120)
fig.subplots_adjust(wspace=0.3)

# Assuming 'names' variable is defined correctly and passed as 'names'
plot_p_value_heatmap(fw_test, name, axes[0], 'A. Forward prediction')
plot_p_value_heatmap(bw_test, name, axes[1], 'B. Backward prediction')

legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label=r'$p \leq 0.05$', markersize=10, markerfacecolor='#A2CEE3'),
    plt.Line2D([0], [0], marker='o', color='w', label=r'$p > 0.05$', markersize=10, markerfacecolor='#D6604D'),
    plt.Line2D([0], [0], marker='o', color='w', label='N/A', markersize=10, markerfacecolor='#D3D3D3')
]

# Position the legend on the figure level, outside the rightmost plot
fig.legend(handles=legend_elements, bbox_to_anchor=(0.5, 0.01), ncols=3, loc='lower center', fontsize=16)

# fig.legend(['R0', 'R1', 'R2', 'R3'], loc='lower center', ncol=2, fontsize=20, bbox_to_anchor=(0.5, 0.001))
fig.subplots_adjust(bottom=0.16) 
plt.show()


In [ ]:
pd.DataFrame(data_list_dict_fw)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_bar_compare(data, ax, title="Values by Type and R Condition", show_values=True):
    """
    Plot a heatmap for the p-value matrix on the provided axes object.

    Parameters:
    - data: List of dictionaries with types and values for R0, R1, R2, R3.
    - ax: Matplotlib axes object to plot the heatmap.
    - title: Optional title for the plot.
    - show_values: Boolean to indicate whether to display the values on the bars.
    """
    types = [item['Type'] for item in data]
    R_labels = ['R0', 'R1', 'R2', 'R3']
    colors = ['#3A8EBA', '#92C5DE', '#F4A582', '#D6604D']
    means = {r: [] for r in R_labels}
    std_devs = {r: [] for r in R_labels}

    # Parse the data to extract means and standard deviations
    for item in data:
        for r in R_labels:
            value, error = item[r].split(' ± ')
            means[r].append(float(value))
            std_devs[r].append(float(error))
    
    # Number of groups and bar width
    n_groups = len(data)
    index = np.arange(n_groups)
    bar_width = 0.2

    # Creating bars for each R
    for i, r in enumerate(R_labels):
        bars = ax.bar(index + i * bar_width, means[r], bar_width, yerr=std_devs[r], 
                      color=colors[i], label=r)

        # Optionally add values on top of the bars
        if show_values:
            for bar in bars:
                height = bar.get_height()
                ax.annotate(f'{height:.1f}',
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom', fontsize=18)

    # Adding labels, title and custom x-axis tick labels, etc.
    #ax.set_xlabel('Type', fontsize=16)
    ax.set_ylabel(r'Average Accuracy (\%)', fontsize=24)
    ax.set_title(title, fontsize = 30, weight = 'bold')
    ax.set_xticks(index + 1.5 * bar_width)
    ax.set_xticklabels(types, rotation=45, fontsize=20)
    ax.set_yticks([0,10,20,30,40])
    ax.set_yticklabels([0,10,20,30,40], fontsize=20)
    ax.legend(fontsize=20)
    

# fig, axes = plt.subplots(1, 2, figsize=(16, 6), dpi=600)
# fig.subplots_adjust(wspace=0.14)

# plot_bar_compare(data_list_dict_fw, axes[0], 'A. Top 1 accuracy - Forward prediction')
# plot_bar_compare(data_list_dict_bw, axes[1], 'B. Top 1 accuracy - Backward prediction')
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for seaborn
sns.set(style="whitegrid")

# Assuming plot_p_value_heatmap is defined elsewhere and it accepts 'ax' as a parameter
# Assuming 'fw_test', 'bw_test', 'data_list_dict_fw', 'data_list_dict_bw', and 'name' are defined elsewhere

# Define legend elements
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label=r'$p \leq 0.05$', markersize=10, markerfacecolor='#A2CEE3'),
    plt.Line2D([0], [0], marker='o', color='w', label=r'$p > 0.05$', markersize=10, markerfacecolor='#D6604D'),
    plt.Line2D([0], [0], marker='o', color='w', label='N/A', markersize=10, markerfacecolor='#D3D3D3')
]

# Create figure and subplots
fig, axes = plt.subplots(2, 2, figsize=(20, 16), dpi=1200)
fig.subplots_adjust(wspace=0.3, hspace=0.3)

# Plot heatmaps on respective axes
plot_bar_compare(data_list_dict_fw, axes[0, 0], 'A. Forward Prediction - Top 1 Accuracy')
plot_bar_compare(data_list_dict_bw, axes[0, 1], 'B. Backward Prediction - Top 1 Accuracy')
plot_p_value_heatmap(fw_test, name, axes[1, 0], 'C. Forward Prediction - T-test Results')
plot_p_value_heatmap(bw_test, name, axes[1, 1], 'D. Backward Prediction - T-test Results')


# Position the legend on the figure level, outside the rightmost plot
fig.legend(handles=legend_elements, bbox_to_anchor=(0.5, 0.01), ncols=3, loc='lower center', fontsize=16, prop={'size':20})

# Adjust the position of the overall figure to make room for the legend
fig.subplots_adjust(bottom=0.12)
fig.savefig('./fig/top_1_temp_compare.pdf', dpi=1200)
# Display the plot
plt.show()


# 3. Time processing

In [ ]:
hier = [3139.097, 3854.369, 5885.342, 6294.58]
good = [2145.046, 14497.126, 137876.927, 307741.2]
h = [3744.996, 22326.635, 169062.646, 361867.284]
non_h = [4348.231, 23538.139, 171314.111, 364311.571]

In [ ]:
from datetime import datetime

# Define the time format from the logs
time_format = "%Y-%m-%d %H:%M:%S,%f"

# Timestamps from the logs
start_time_str = "2024-06-14 15:56:44,503"
end_time_str = "2024-06-18 21:08:36,074"

# Convert string to datetime
start_time = datetime.strptime(start_time_str, time_format)
end_time = datetime.strptime(end_time_str, time_format)

# Calculate difference in seconds
time_difference = end_time - start_time
total_seconds = time_difference.total_seconds()

print("Total time in seconds:", total_seconds)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Enable LaTeX rendering in matplotlib
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')  # Ensure amsmath is loaded


def plot_processing_times(times):
    # Convert to hours
    for key in times:
        times[key] = np.array(times[key]) / 3600

    # Stages
    stages = ['R0', 'R1', 'R2', 'R3']

    # Create a DataFrame
    df = pd.DataFrame(times, index=stages).reset_index().melt(id_vars='index', var_name='Method', value_name='Time (hours)')
    df.rename(columns={'index': 'Stage'}, inplace=True)

    # Create the plot
    plt.figure(figsize=(12, 8))
    palette = sns.color_palette("coolwarm", n_colors=len(times.keys()))
    bar_plot = sns.barplot(x='Stage', y='Time (hours)', hue='Method', data=df, palette=palette)

    plt.title('Processing Time across Different Methods by Radius', fontsize=16)
    plt.xlabel(None)
    plt.ylabel('Time (Hours)', fontsize=14)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(title='Rule Type', title_fontsize='16', fontsize='14', loc='upper left', bbox_to_anchor=(0.01, 1))

    # Add text annotations on the bars
    for p in bar_plot.patches:
        bar_height = p.get_height()
        if bar_height > 0.01:  # Adjust this threshold as needed
            bar_plot.annotate(format(p.get_height(), '.1f'), 
                            (p.get_x() + p.get_width() / 2., p.get_height()), 
                            ha = 'center', va = 'center', 
                            xytext = (0, 9), 
                            textcoords = 'offset points', fontsize=14)

    plt.grid(True, linestyle='--', alpha=0.6)  # Optional: add grid for better readability
    plt.tight_layout()  # Adjust layout to make room for the legend
    plt.show()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Enable LaTeX rendering in matplotlib
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')  # Ensure amsmath is loaded


def plot_processing_times(times, ax=None):
    # Convert to hours
    for key in times:
        times[key] = np.array(times[key]) / 3600

    # Stages
    stages = [r'$R_{0}$', r'$R_{1}$', r'$R_{2}$', r'$R_{3}$']

    # Create a DataFrame
    df = pd.DataFrame(times, index=stages).reset_index().melt(id_vars='index', var_name='Method', value_name='Time (hours)')
    df.rename(columns={'index': 'Stage'}, inplace=True)

    # Create the plot on the provided ax
    if ax is None:
        ax = plt.gca()  # Get current axis if not provided

    palette = sns.color_palette("coolwarm", n_colors=len(times.keys()))
    bar_plot = sns.barplot(x='Stage', y='Time (hours)', hue='Method', data=df, palette=palette, ax=ax)

    ax.set_title('A. Processing Time across Different Methods by Radius', fontsize=20, weight = 'bold')
    ax.set_xlabel(None)
    ax.set_ylabel('Time (Hours)', fontsize=18)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=18)
    ax.set_yticks(ax.get_yticks())
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=18)
    ax.legend(title='Rule Type', title_fontsize='20', fontsize='18', loc='upper left', bbox_to_anchor=(0.01, 1))

    # Add text annotations on the bars
    for p in bar_plot.patches:
        bar_height = p.get_height()
        if bar_height > 0.01:  # Adjust this threshold as needed
            if p.get_height() < 100:
                bar_plot.annotate(format(p.get_height(), '.1f'), 
                                (p.get_x() + p.get_width() / 2., p.get_height()), 
                                ha='center', va='center', 
                                xytext=(0, 9), 
                                textcoords='offset points', fontsize=16)
            else:
                if p.get_height() < 101:
                    #p.get_height() = 100
                    bar_plot.annotate(format(100, '.0f'), 
                                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                                    ha='center', va='center', 
                                    xytext=(0, 9), 
                                    textcoords='offset points', fontsize=16)
                else:
                    bar_plot.annotate(format(p.get_height(), '.0f'), 
                                (p.get_x() + p.get_width() / 2., p.get_height()), 
                                ha='center', va='center', 
                                xytext=(0, 9), 
                                textcoords='offset points', fontsize=16)

    ax.grid(True, linestyle='--', alpha=0.6)  # Optional: add grid for better readability
    #plt.tight_layout()  # Adjust layout to make room for the legend

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import auc

def plot_combined_roc(forward_data_list, backward_data_list, ax):
    # Set the Seaborn style
    sns.set(style="whitegrid")

    # Colors and markers for better distinction
    forward_color = '#3A8EBA'
    backward_color = '#D6604D'
    markers = ['o', '^', 's', 'p']  # Different marker for each R value
    marker_labels = [r'$R_{0}$', r'$R_{1}$', r'$R_{2}$', r'$R_{3}$']

    # Function to plot ROC curves for a given data list and axis
    def plot_roc(data_list, ax, direction, color):
        for data in data_list:
            template_type = data['Type']
            reconstruction_rates = data['reconstruction_rates']
            number_of_solutions = data['number_of_solutions']
            
            # Normalize the reconstruction rates (converting to proportions)
            TPR = [x / 100 for x in reconstruction_rates]  # True Positive Rate

            # Applying Min-Max scaling to number_of_solutions to calculate FPR
            min_solution = min(number_of_solutions)
            max_solution = max(number_of_solutions)
            FPR = [(x - min_solution) / (max_solution - min_solution) for x in number_of_solutions]  # False Positive Rate
            
            # Calculate the AUC
            roc_auc = auc(FPR, TPR)
            
            # Plot ROC curve for the current template type
            ax.plot(FPR, TPR, linestyle='-', color=color,
                    label=f'{template_type} ({direction}) (AUC = {roc_auc:.2f})')
            
            # Plot each point with a different marker
            for i, (fpr, tpr) in enumerate(zip(FPR, TPR)):
                ax.plot(fpr, tpr, marker=markers[i], color=color)
            # # Add data point markers and labels
            # for i, (fpr, tpr) in enumerate(zip(FPR, TPR)):
            #     ax.plot(fpr, tpr, marker=markers[i % len(markers)], color=color)
            #     ax.annotate(fr'$R_{i}$', (fpr, tpr), textcoords="offset points", xytext=(0,10), ha='center')

    # Plot forward predictions
    plot_roc(forward_data_list, ax, 'Forward', forward_color)
    
    # Plot backward predictions
    plot_roc(backward_data_list, ax, 'Backward', backward_color)
    
    # Create a legend for the markers
    for i, marker_label in enumerate(marker_labels):
        ax.plot([], [], marker=markers[i], color='gray', label=marker_label, linestyle='None')

    # Enhance the plot aesthetics
    ax.set_xlabel('FPR', fontsize=18)
    ax.set_ylabel(r'Reconstruction  rate $\mathcal(R)$', fontsize=18)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=18)
    ax.set_yticks(ax.get_yticks())
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=18)
    ax.legend(loc='lower right', ncol=3, fontsize=16, title_fontsize='18')
    ax.set_title('B. Foward and Backward Performance by Radius', fontsize=20, weight = 'bold')
    #sns.despine(ax=ax)
    ax.grid(True)
    plt.tight_layout()


In [ ]:
times_compare = {
        r'$Q_{\text{hier}}$': [3139.097, 3854.369, 5885.342, 6294.58],
        r'$Q_{\text{cyclic}}$': [2145.046, 14497.126, 137876.927, 307741.2],
        r'$Q_{\text{complete}}$': [3744.996, 22326.635, 169062.646, 361867.284], 
        r'$Q_{\text{raw}}$': [4348.231, 23538.139, 171314.111, 364311.571]
    }


fig, axs = plt.subplots(1, 2, figsize=(16, 8))
plot_processing_times(times_compare, ax=axs[0])
# Plot on the first subplot
plot_combined_roc(fw_list_dict[-1:], bw_list_dict[-1:], ax=axs[1])


plt.subplots_adjust(wspace=0.18)
plt.savefig('./fig/time_roc.pdf', dpi = 600)
# Display the plot
plt.show()

In [ ]:
fw_list_dict[-1:], bw_list_dict[-1:]

In [ ]:
import numpy as np

def find_optimal_cutoff(data):
    optimal_cutoffs = []

    for direction in data:
        gmeans = []
        best_index = 0
        best_gmean = 0
        
        for i in range(len(direction['reconstruction_rates'])):
            reconstruction_rate = direction['reconstruction_rates'][i]/100
            fpr = direction['number_of_solutions'][i]
            
            # Compute Gmean
            gmean = np.sqrt(reconstruction_rate * (1 - fpr))
            gmeans.append(gmean)
            
            if gmean > best_gmean:
                best_gmean = gmean
                best_index = i
        
        optimal_cutoffs.append({
            'Type': direction['Type'],
            'optimal_cutoff': best_index,
            'reconstruction_rate': direction['reconstruction_rates'][best_index],
            'fpr': direction['number_of_solutions'][best_index],
            'gmean': best_gmean,
            'gmeans': gmeans
        })

    return optimal_cutoffs

# Example data
forward_data = [{'Type': '$Q_{\\text{hier}}$', 'reconstruction_rates': [78.93, 78.19, 74.67, 66.21], 'number_of_solutions': [0.94, 0.79, 0.61, 0.59]}]
backward_data = [{'Type': '$Q_{\\text{hier}}$', 'reconstruction_rates': [78.93, 78.21, 74.77, 66.35], 'number_of_solutions': [0.94, 0.79, 0.73, 0.7]}]

# Finding optimal cutoffs
forward_optimal_cutoff = find_optimal_cutoff(forward_data)
backward_optimal_cutoff = find_optimal_cutoff(backward_data)

print("Forward Optimal Cutoff:", forward_optimal_cutoff)
print("Backward Optimal Cutoff:", backward_optimal_cutoff)

In [ ]:
forward_data_new = [{'Type': '$Q_{\\text{hier}}$', 'reconstruction_rates': [78.93, 78.19, 74.67, 66.21], 'number_of_solutions': [0.94, 0.79, 0.61, 0.59], 'gmeans': [2.176189329998657, 4.052147578753765, 5.396415476962463, 5.210191935044236]}]
backward_data_new = [{'Type': '$Q_{\\text{hier}}$', 'reconstruction_rates': [78.93, 78.21, 74.77, 66.35], 'number_of_solutions': [0.94, 0.79, 0.73, 0.7], 'gmeans': [2.176189329998657, 4.05266578932929, 4.493094701873086, 4.461501989240843]}]

# Sorting Function

In [ ]:
import random
import numpy as np
from joblib import Parallel, delayed
from SynTemp.SynRule.rule_benchmark import RuleBenchmark
from SynTemp.SynChemistry.sf_similarity import SFSimilarity
from typing import List, Dict, Tuple

def _evaluation(data: List[Dict], ground_truth_key: str = 'reactions', rank_list_key: str = "ranked_reactions", ignore_stereo: bool = True, n_jobs: int = 4) -> Tuple[float, float, List[float]]:
    """
    Evaluate the performance of a ranking system using random seeds in parallel.

    Parameters:
    - data: List of dictionaries containing the data.
    - ground_truth_key: The key for the ground truth reactions. Default is 'reactions'.
    - rank_list_key: The key for the ranked reactions list. Default is 'ranked_reactions'.
    - ignore_stereo: Boolean flag to ignore stereo chemistry. Default is True.
    - n_jobs: The number of jobs to run in parallel. Default is 4.

    Returns:
    - A tuple containing:
      - Mean accuracy of the evaluations.
      - Standard deviation of the accuracies.
      - List of individual evaluation results.
    """
    if not data:
        raise ValueError("Data list is empty.")
    
    # Define the function to compute TopKAccuracy
    def compute_topk_accuracy(k):
        return RuleBenchmark.TopKAccuracy(data, ground_truth_key, rank_list_key, k, ignore_stereo, scoring_function=SFSimilarity(['ECFP6']))

    # Define the list of k values
    k_values = [1, 3, 5, 10]

    # Use joblib to parallelize the computation
    results = Parallel(n_jobs=n_jobs, verbose=2)(delayed(compute_topk_accuracy)(k) for k in k_values)

    return results

In [ ]:
list_data_fw = [[nh_r0_fw, nh_r1_fw, nh_r2_fw, nh_r3_fw], 
                [h_r0_fw, h_r1_fw, h_r2_fw, h_r3_fw],
                [gh_r0_fw, gh_r1_fw, gh_r2_fw, gh_r3_fw],
                [hier_r0_fw, hier_r1_fw, hier_r2_fw, hier_r3_fw]]
data_list_dict_fw = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

data_list_dict_bw = [{'Type':r'$Q_{\text{raw}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{complete}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                     {'Type':r'$Q_{\text{cyclic}}$', "R0":[], "R1":[], "R2":[], "R3":[]}, 
                    {'Type':r'$Q_{\text{hier}}$', "R0":[], "R1":[], "R2":[], "R3":[]}]

In [ ]:
for key, data in enumerate(list_data_fw):
    for radius, value in enumerate(data):
        result = _evaluation(value)
        data_list_dict_fw[key][f'R{radius}'] = result

In [ ]:
data_list_dict_fw

In [ ]:
for key, data in enumerate(list_data_bw):
    for radius, value in enumerate(data):
        result = _evaluation(value)
        data_list_dict_bw[key][f'R{radius}'] = result

In [ ]:
data_list_dict_bw